In [1]:
import pandas as pd
import numpy as np
import json, graphviz, sys
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [26]:
loans = pd.read_csv('lending-club-data.csv', low_memory=False)

loans['bad_loans'].value_counts()
loans['safe_loans'] = loans['bad_loans'].apply(lambda x: 1-x)
del loans['bad_loans']

target = 'safe_loans'
features = ['grade',                     # grade of the loan (categorical)
            'sub_grade_num',             # sub-grade of the loan as a number from 0 to 1
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'payment_inc_ratio',         # ratio of the monthly payment to income
            'delinq_2yrs',               # number of delinquincies
            'delinq_2yrs_zero',          # no delinquincies in last 2 years
            'inq_last_6mths',            # number of creditor inquiries in last 6 months
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'open_acc',                  # number of open credit accounts
            'pub_rec',                   # number of derogatory public records
            'pub_rec_zero',              # no derogatory public records
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
            'int_rate',                  # interest rate of the loan
            'total_rec_int',             # interest received to date
            'annual_inc',                # annual income of borrower
            'funded_amnt',               # amount committed to the loan
            'funded_amnt_inv',           # amount committed by investors for the loan
            'installment',               # monthly payment owed by the borrower
           ]
loans = loans[features + [target]]

In [30]:
loans = loans.dropna()

In [31]:
# one hot encode
for name, dtype in zip(loans.dtypes.index, loans.dtypes.values):
    if dtype == 'object':
        onehot = pd.get_dummies(loans[name])
        onehot.rename(columns={x:name+'_'+x.strip().replace(' ', '_') for x in onehot.columns}
                  , inplace=True)
        loans = loans.join(onehot)
        del loans[name]

In [32]:
feature_onehot = [x for x in loans.columns if x != target]

In [33]:
with open('module-8-assignment-1-train-idx.json', 'r') as f:
    train_idx = json.load(f)
with open('module-8-assignment-1-validation-idx.json', 'r') as f:
    valid_idx = json.load(f)

train_data = loans.iloc[train_idx, :].reset_index(drop=True)
validation_data = loans.iloc[valid_idx, :].reset_index(drop=True)

In [37]:
train_data.head()

,sub_grade_num,short_emp,emp_length_num,dti,payment_inc_ratio,delinq_2yrs,delinq_2yrs_zero,inq_last_6mths,last_delinq_none,last_major_derog_none,...,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding
0,0.8,1,1,1.00,2.39320,0.0,1.0,5.0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0.4,0,5,5.55,4.57170,0.0,1.0,2.0,1,1,...,0,0,0,0,0,0,0,1,0,0
2,1.0,1,1,18.08,9.71600,0.0,1.0,0.0,1,1,...,0,0,0,0,0,0,1,0,0,0
3,0.2,1,1,10.08,12.21520,0.0,1.0,1.0,1,1,...,1,0,0,0,0,0,0,0,0,0
4,0.4,0,4,7.06,3.90888,0.0,1.0,2.0,1,1,...,0,0,0,0,0,0,1,0,0,0


In [8]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0    
    # Count the number of 1's (safe loans)
    positive_counts = sum(labels_in_node == 1)    
    # Count the number of -1's (risky loans)
    negative_counts = len(labels_in_node) - positive_counts                
    # Return the number of mistakes that the majority classifier makes.
    return (min(positive_counts, negative_counts))

In [9]:
def best_splitting_feature(data, features, target):
    
    target_values = data[target]
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        right_split =  data[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        left_mistakes = intermediate_node_num_mistakes(left_split[target])             

        # Calculate the number of misclassified examples in the right split.
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        error = (left_mistakes + right_mistakes) / num_data_points

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        if error < best_error:
            best_error = error
            best_feature = feature
    
    return best_feature # Return the best feature we found

In [10]:
def create_leaf(target_values):    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True    }    
   
    # Count the number of data points that are +1 and -1 in this node.
    num_positive = len(target_values[target_values == 1])
    num_negative = len(target_values[target_values == 0])    

    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or 0) in leaf['prediction']
    if num_positive > num_negative:
        leaf['prediction'] = 1       
    else:
        leaf['prediction'] = 0              

    # Return the leaf node
    return leaf 

def reached_minimum_node_size(data, min_node_size):
    # Return True if the number of data points is less than or equal to the minimum node size.
    return (len(data) <= min_node_size)

def error_reduction(error_before_split, error_after_split):
    # Return the error before the split minus the error after the split.
    return error_before_split - error_after_split

In [62]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0,
                         verbose=True):
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    if verbose:
        print("--------------------------------------------------------------------")
        print("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    
    # Stopping condition 1
    # (Check if there are mistakes at current node.
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.)
    if intermediate_node_num_mistakes(target_values) == 0:  
        if verbose: print("Stopping condition 1 reached.")     
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if len(remaining_features) == 0:
        if verbose: print("Stopping condition 2 reached.")    
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Early stopping condition 1: Reached max depth limit.
    if current_depth >= max_depth:  
        if verbose: print("Reached maximum depth. Stopping for now.")
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)
    
    # Early stopping condition 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    if reached_minimum_node_size(data, min_node_size):          
        if verbose: print("Early stopping condition 2 reached. Reached minimum node size.")
        return create_leaf(target_values)

    # Find the best splitting feature (recall the function best_splitting_feature 
    # implemented above)
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    # Early stopping condition 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_mistakes = intermediate_node_num_mistakes(left_split[target])
    right_mistakes = intermediate_node_num_mistakes(right_split[target])
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    if error_reduction(error_before_split, error_after_split) <= min_error_reduction:
        if verbose: 
            print("Early stopping condition 3 reached. Minimum error reduction.")
        return create_leaf(target_values)
    
    remaining_features.remove(splitting_feature)
    if verbose:
        print("Split on feature %s. (%s, %s)" % (
                      splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        if verbose: print("Creating a left leaf node.")
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        if verbose: print("Creating a right leaf node.")
        return create_leaf(right_split[target])
        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, 
                                     current_depth + 1, max_depth, 
                                     min_node_size, min_error_reduction, verbose)        
    right_tree = decision_tree_create(right_split, remaining_features, target, 
                                      current_depth + 1, max_depth, 
                                      min_node_size, min_error_reduction, verbose)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [29]:
my_decision_tree_new = decision_tree_create(train_data, feature_onehot, target, 
                                        current_depth=0, max_depth=6,
                                        min_node_size = 100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_36_months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature emp_length_n/a. (96, 5)
--------------------------------------------------------------------
Subtree, depth = 3 (96 data points).
Early stopping condition 2 reached. Reached minimum node size.
--------------------------------------------------------------------
Subtree, depth = 3 (5 data points).
Early stopping condition 2 reached. Reached minimum node size.
--------------------------------------------

In [25]:
my_decision_tree_old = decision_tree_create(train_data, feature_onehot, 'safe_loans', max_depth = 6,
                                            min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_36_months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Re

In [39]:
def classify(tree, x, path, annotate=False):
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
             print("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction'], path
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, path+1, annotate)
        else:
            return classify(tree['right'], x, path+1, annotate)

In [46]:
# print(validation_data.iloc[0, :])
print('Predicted class: {}, path length: {}'.format(*classify(my_decision_tree_new, 
                                        validation_data.iloc[0, :],
                                        path=0, annotate=True)))

Split on term_36_months = 0
Split on grade_A = 0
At leaf, predicting 0
Predicted class: 0, path length: 2


In [47]:
# print(validation_data.iloc[0, :])
print('Predicted class: {}, path length: {}'.format(*classify(my_decision_tree_old, 
                                        validation_data.iloc[0, :],
                                        path=0, annotate=False)))

Predicted class: 0, path length: 5


In [51]:
long, equal, short = 0, 0, 0
max_len = 0
for i in range(validation_data.shape[0]):
    path1 = classify(my_decision_tree_new, 
                                        validation_data.iloc[i, :],
                                        path=0, annotate=False)[1]
    path2 = classify(my_decision_tree_old, 
                                        validation_data.iloc[i, :],
                                        path=0, annotate=False)[1]
    max_len = max(path1, path2, max_len)
    if path1 > path2:
        long += 1
    elif path1 < path2:
        short += 1
    else:
        equal += 1

print(long, equal, short)
print(max_len)

0 6957 2327
6


In [79]:
def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    prediction = []
    for i in range(len(data)):
        prediction.append(classify(tree, data.iloc[i, :], 0)[0])
    
    # Once you've made the predictions, calculate the classification error and return it
    acc = sum(prediction == data[target]) / len(data)
    
    return acc

In [76]:
evaluate_classification_error(my_decision_tree_new, validation_data)

0.61632916846186991

In [77]:
evaluate_classification_error(my_decision_tree_old, validation_data)

0.61622145626884961

In [66]:
model_1 = decision_tree_create(train_data, feature_onehot, 'safe_loans', max_depth=2,
                            min_node_size=0, min_error_reduction=-1, verbose=False)
model_2 = decision_tree_create(train_data, feature_onehot, 'safe_loans', max_depth=6,
                            min_node_size=0, min_error_reduction=-1, verbose=False)

In [67]:
%%time
model_3 = decision_tree_create(train_data, feature_onehot, 'safe_loans', max_depth=14,
                            min_node_size=0, min_error_reduction=-1, verbose=False)

CPU times: user 13.3 s, sys: 64.1 ms, total: 13.3 s
Wall time: 13.4 s


In [80]:
print("Training data, classification error (model 1):", 
      evaluate_classification_error(model_1, train_data))
print("Training data, classification error (model 2):", 
      evaluate_classification_error(model_2, train_data))
print("Training data, classification error (model 3):", 
      evaluate_classification_error(model_3, train_data))

Training data, classification error (model 1): 0.599962389856
Training data, classification error (model 2): 0.618149580916
Training data, classification error (model 3): 0.623817966903


In [90]:
print("Validation data, classification error (model 1):", 
      evaluate_classification_error(model_1, validation_data))
print("Validation data, classification error (model 2):", 
      evaluate_classification_error(model_2, validation_data))
print("Validataion data, classification error (model 3):", 
      evaluate_classification_error(model_3, validation_data))

Validation data, classification error (model 1): 0.601895734597
Validation data, classification error (model 2): 0.616221456269
Validataion data, classification error (model 3): 0.62268418785


In [82]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

In [83]:
print(count_leaves(model_1), count_leaves(model_2), count_leaves(model_3))

4 19 41


In [84]:
model_4 = decision_tree_create(train_data, feature_onehot, 'safe_loans', max_depth=6,
                            min_node_size=0, min_error_reduction=-1, verbose=False)
model_5 = decision_tree_create(train_data, feature_onehot, 'safe_loans', max_depth=6,
                            min_node_size=0, min_error_reduction=0, verbose=False)
model_6 = decision_tree_create(train_data, feature_onehot, 'safe_loans', max_depth=6,
                            min_node_size=0, min_error_reduction=5, verbose=False)

In [85]:
print("Validation data, classification error (model 4):", 
      evaluate_classification_error(model_4, validation_data))
print("Validation data, classification error (model 5):", 
      evaluate_classification_error(model_5, validation_data))
print("Validataion data, classification error (model 6):", 
      evaluate_classification_error(model_6, validation_data))

Validation data, classification error (model 4): 0.616221456269
Validation data, classification error (model 5): 0.616221456269
Validataion data, classification error (model 6): 0.496553209823


In [86]:
print(count_leaves(model_4), count_leaves(model_5), count_leaves(model_6))

19 13 1


In [87]:
model_7 = decision_tree_create(train_data, feature_onehot, 'safe_loans', max_depth=6,
                            min_node_size=0, min_error_reduction=-1, verbose=False)
model_8 = decision_tree_create(train_data, feature_onehot, 'safe_loans', max_depth=6,
                            min_node_size=2000, min_error_reduction=-1, verbose=False)
model_9 = decision_tree_create(train_data, feature_onehot, 'safe_loans', max_depth=6,
                            min_node_size=50000, min_error_reduction=-1, verbose=False)

In [88]:
print("Validation data, classification error (model 7):", 
      evaluate_classification_error(model_7, validation_data))
print("Validation data, classification error (model 8):", 
      evaluate_classification_error(model_8, validation_data))
print("Validataion data, classification error (model 9):", 
      evaluate_classification_error(model_9, validation_data))

Validation data, classification error (model 7): 0.616221456269
Validation data, classification error (model 8): 0.615467470918
Validataion data, classification error (model 9): 0.496553209823


In [89]:
print(count_leaves(model_7), count_leaves(model_8), count_leaves(model_9))

19 12 1
